<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/CIFAR-100/CIFAR_100_CNN_Efficient_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 순서
# 일반적으로 수행, finetuning, freeze 후 분류기 학습.
# dropout 추가 
# 정규화
# augmentaion

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
import time
import os
import copy

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print("기기 사용 {}".format(device))

기기 사용 cuda


In [3]:
transform = transforms.Compose([
                                transforms.ToTensor()
])

In [4]:
trainset = torchvision.datasets.CIFAR100(root = './cifar-100',
                                         train = True,
                                         download = True,
                                         transform = transform)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./cifar-100/cifar-100-python.tar.gz to ./cifar-100


In [5]:
print(trainset.data.shape)

(50000, 32, 32, 3)


In [6]:
train_data_mean = trainset.data.mean(axis = (0,1,2)) # 각 축에대한 평균 
train_data_std = trainset.data.std(axis = (0,1,2)) # 각 축에 대한 분산 ()

print(f"train_mean : {train_data_mean}")
print(f"train_std : {train_data_std}")

train_data_mean = (train_data_mean/ 255.)
train_data_std = (train_data_std/ 255.)

print(train_data_mean)
print(train_data_std)


train_mean : [129.30416561 124.0699627  112.43405006]
train_std : [68.1702429  65.39180804 70.41837019]
[0.50707516 0.48654887 0.44091784]
[0.26733429 0.25643846 0.27615047]


In [7]:
transform_train = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(train_data_mean , train_data_std)
])


train_set = torchvision.datasets.CIFAR100(
    root = './cifar-100',
    train = True,
    download = True,
    transform = transform_train
)

test_set = torchvision.datasets.CIFAR100(
    root = './cifar-100',
    train = False,
    download = True,
    transform = transform_test 
)

train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size = 256,
                                           shuffle = True,
                                           num_workers = 0)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size= 256,
                                          num_workers = 0)



Files already downloaded and verified
Files already downloaded and verified


In [8]:
import torchvision.models as models

In [10]:
test_model = models.efficientnet_b7(pretrained= True)

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth


  0%|          | 0.00/255M [00:00<?, ?B/s]

In [12]:
test_model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivatio

In [16]:
test_model.classifier[1] = nn.Linear(2560, 100)

In [ ]:
test_model

In [64]:
test_model.fc = nn.Linear(2560, 100) #test_model.fc = nn.Linear(2048, 1024)

In [13]:
class mymodel(nn.Module):
  def __init__(self):
    super(mymodel, self).__init__()
    self.linear1 = nn.Linear(1024, 512)
    self.linear2 = nn.Linear(512, 256)
    self.linear3 = nn.Linear(256, 100)
    self.relu = nn.ReLU(inplace= True)
    
    torch.nn.init.xavier_uniform_(self.linear1.weight)
    torch.nn.init.xavier_uniform_(self.linear2.weight)
    torch.nn.init.xavier_uniform_(self.linear3.weight)
    
    self.layer1 = torch.nn.Sequential(
        self.linear1,
        self.relu,
        self.linear2,
        self.relu,
        self.linear3
    )
    

  def forward(self, x):
    out = self.layer1(x)

    return out
    
  


In [14]:
my_model = mymodel()

In [15]:
my_model

mymodel(
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=100, bias=True)
  )
)

In [16]:
total_model = nn.Sequential(
    test_model,
    my_model
)

In [18]:
test_model = test_model.to(device)  #total_model = total_model.to(device)

In [19]:
test_image = torch.rand(256,3,32,32).to(device)

print(total_model(test_image).shape)

NameError: ignored

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(test_model.parameters(), lr = 0.001)  #optimizer = optim.Adam(total_model.parameters(), lr = 0.001)


In [21]:
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 4, gamma= 0.3)

In [22]:
for epoch in range(10):
  avg_cost = 0
  total_batch = len(train_loader) 
  acc = 0
  for x , y in train_loader:
    optimizer.zero_grad()
    x = x.to(device)
    y = y.to(device)
    
    h = test_model(x) #total_model(x)
    
    correct_predict = torch.argmax(h, 1)

    loss = criterion(h , y)
    loss.backward()
    
    optimizer.step()
    
    acc += correct_predict.eq(y.view_as(correct_predict)).sum().item()
    
    avg_cost += loss / total_batch
  
  lr_scheduler.step()
  print("epoch: {} , cost = {:.9f}, acc: {}/{} ({:.2f}%)".format(epoch, avg_cost, acc, len(train_loader.dataset), (acc/len(train_loader.dataset)) ))

epoch: 0 , cost = 3.091597080, acc: 12083/50000 (0.24%)
epoch: 1 , cost = 1.885493517, acc: 24021/50000 (0.48%)
epoch: 2 , cost = 1.449324489, acc: 29323/50000 (0.59%)
epoch: 3 , cost = 1.157368422, acc: 33209/50000 (0.66%)
epoch: 4 , cost = 0.657503724, acc: 39905/50000 (0.80%)
epoch: 5 , cost = 0.439337492, acc: 43097/50000 (0.86%)
epoch: 6 , cost = 0.316205978, acc: 45004/50000 (0.90%)
epoch: 7 , cost = 0.237503424, acc: 46214/50000 (0.92%)
epoch: 8 , cost = 0.133609429, acc: 47949/50000 (0.96%)
epoch: 9 , cost = 0.091184326, acc: 48667/50000 (0.97%)


In [24]:
with torch.no_grad():
  test_model.eval()
  acc =0 
  avg_test_loss = 0
  correct = 0
  for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    
    prediction = test_model(data) #total_model(day)
    
    test_loss = criterion(prediction, target) 
    
    correct_predict = torch.argmax(prediction, 1)

    avg_test_loss += test_loss / len(test_loader)

    acc += correct_predict.eq(target.view_as(correct_predict)).sum().item()

  
  print("acc: {}/{} ({:.2f}%), Test_loss : {}".format(acc, len(test_loader.dataset), (acc/len(test_loader.dataset)) * 100. , avg_test_loss))


acc: 6149/10000 (61.49%), Test_loss : 2.03320574760437


In [11]:
from torchsummary import summary as summary_
summary_(test_model, (3,32,32), batch_size = 256)

RuntimeError: ignored